In [4]:
import re
import json
import os

# --- 1. Object-Oriented Programming (OOP) ---
# Naming Convention: Class names use CamelCase (e.g., Contact, ContactManager)
# Variable and function names use snake_case (e.g., first_name, add_contact)

class Contact:
    """
    Represents a single contact with encapsulation.
    """
    def __init__(self, name, phone, email, contact_id=None):
        # Encapsulation: Using private-like attributes (conventionally with _)
        # and public getter/setter methods.
        if not name:
            raise ValueError("Nama tidak boleh kosong.")
        if not self._validate_phone(phone):
            raise ValueError("Format nomor telepon tidak valid. Gunakan format angka saja.")
        if not self._validate_email(email):
            raise ValueError("Format email tidak valid.")

        self._name = name
        self._phone = phone
        self._email = email
        self._contact_id = contact_id if contact_id else self._generate_id()

    def _generate_id(self):
        """Generates a simple unique ID for the contact."""
        # In a real application, this would be more robust (e.g., UUID)
        return f"C-{abs(hash(self._name + self._phone)) % (10**6)}"

    # Getter methods (Encapsulation)
    def get_name(self):
        return self._name

    def get_phone(self):
        return self._phone

    def get_email(self):
        return self._email

    def get_id(self):
        return self._contact_id

    # Setter methods with validation (Encapsulation)
    def set_name(self, name):
        if not name:
            raise ValueError("Nama tidak boleh kosong.")
        self._name = name

    def set_phone(self, phone):
        if not self._validate_phone(phone):
            raise ValueError("Format nomor telepon tidak valid. Gunakan format angka saja.")
        self._phone = phone

    def set_email(self, email):
        if not self._validate_email(email):
            raise ValueError("Format email tidak valid.")
        self._email = email

    # --- Regular Expression ---
    def _validate_phone(self, phone):
        """Validates phone number format using regex (digits only, optional + at start)."""
        # Regex: ^\+?\d+$
        # ^      : Start of the string
        # \+?    : Optional '+' character
        # \d+    : One or more digits
        # $      : End of the string
        return re.fullmatch(r"^\+?\d+$", phone) is not None

    def _validate_email(self, email):
        """Validates email format using a common regex pattern."""
        # Regex: ^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$
        # This is a common pattern, but email validation can be complex.
        return re.fullmatch(r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$", email) is not None

    def to_dict(self):
        """Converts contact object to a dictionary for saving."""
        return {
            "id": self._contact_id,
            "name": self._name,
            "phone": self._phone,
            "email": self._email
        }

    @classmethod
    def from_dict(cls, data):
        """Creates a Contact object from a dictionary."""
        return cls(data["name"], data["phone"], data["email"], data["id"])

    def __str__(self):
        return f"ID: {self._contact_id}, Nama: {self._name}, Telepon: {self._phone}, Email: {self._email}"

# Inheritance Example (optional, but demonstrates the concept)
class FamilyContact(Contact):
    """Represents a family contact with an additional relationship attribute."""
    def __init__(self, name, phone, email, relationship, contact_id=None):
        super().__init__(name, phone, email, contact_id)
        if not relationship:
            raise ValueError("Hubungan tidak boleh kosong untuk kontak keluarga.")
        self._relationship = relationship

    def get_relationship(self):
        return self._relationship

    def set_relationship(self, relationship):
        if not relationship:
            raise ValueError("Hubungan tidak boleh kosong untuk kontak keluarga.")
        self._relationship = relationship

    def to_dict(self):
        data = super().to_dict()
        data["relationship"] = self._relationship
        return data

    @classmethod
    def from_dict(cls, data):
        return cls(data["name"], data["phone"], data["email"], data["relationship"], data["id"])

    def __str__(self):
        return f"{super().__str__()}, Hubungan: {self._relationship}"


class ContactManager:
    """
    Manages a collection of contacts, including saving and loading.
    """
    def __init__(self, filename="contacts.json"):
        self.filename = filename
        self.contacts = []
        self._load_contacts()

    # --- File Handling ---
    def _load_contacts(self):
        """Loads contacts from the JSON file."""
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r') as f:
                    data = json.load(f)
                    self.contacts = [Contact.from_dict(d) for d in data]
                print(f"Kontak berhasil dimuat dari '{self.filename}'.")
            except json.JSONDecodeError:
                print(f"Peringatan: File '{self.filename}' rusak atau kosong. Memulai dengan daftar kosong.")
                self.contacts = []
            except Exception as e:
                print(f"Terjadi kesalahan saat memuat kontak: {e}")
                self.contacts = []
        else:
            print(f"File '{self.filename}' tidak ditemukan. Memulai dengan daftar kosong.")

    def _save_contacts(self):
        """Saves current contacts to the JSON file."""
        try:
            with open(self.filename, 'w') as f:
                json.dump([c.to_dict() for c in self.contacts], f, indent=4)
            print(f"Kontak berhasil disimpan ke '{self.filename}'.")
        except Exception as e:
            print(f"Terjadi kesalahan saat menyimpan kontak: {e}")

    def add_contact(self, contact):
        """Adds a new contact to the manager."""
        self.contacts.append(contact)
        self._save_contacts()
        print(f"Kontak '{contact.get_name()}' berhasil ditambahkan.")

    def view_contacts(self):
        """Displays all contacts."""
        if not self.contacts:
            print("Daftar kontak kosong.")
            return
        print("\n--- Daftar Kontak ---")
        for contact in self.contacts:
            print(contact)
        print("--------------------")

    def find_contact(self, query):
        """Finds contacts by name, phone, or email."""
        found_contacts = [
            c for c in self.contacts
            if query.lower() in c.get_name().lower() or
               query in c.get_phone() or
               query.lower() in c.get_email().lower()
        ]
        if not found_contacts:
            print(f"Tidak ada kontak ditemukan untuk '{query}'.")
            return []
        print(f"\n--- Kontak Ditemukan untuk '{query}' ---")
        for contact in found_contacts:
            print(contact)
        print("----------------------------")
        return found_contacts

    def delete_contact(self, contact_id):
        """Deletes a contact by ID."""
        original_count = len(self.contacts)
        self.contacts = [c for c in self.contacts if c.get_id() != contact_id]
        if len(self.contacts) < original_count:
            self._save_contacts()
            print(f"Kontak dengan ID '{contact_id}' berhasil dihapus.")
        else:
            print(f"Kontak dengan ID '{contact_id}' tidak ditemukan.")

# --- Main Program / User Interface ---
def main():
    manager = ContactManager("my_contacts.json") # File akan dibuat jika belum ada

    while True:
        print("\n--- Menu Manajemen Kontak ---")
        print("1. Tambah Kontak")
        print("2. Lihat Semua Kontak")
        print("3. Cari Kontak")
        print("4. Hapus Kontak")
        print("5. Keluar")
        print("-----------------------------")

        choice = input("Pilih opsi (1-5): ")

        if choice == '1':
            print("\n--- Tambah Kontak Baru ---")
            name = input("Nama: ")
            phone = input("Telepon: ")
            email = input("Email: ")
            contact_type = input("Tipe kontak (biasa/keluarga): ").lower()

            try:
                if contact_type == "keluarga":
                    relationship = input("Hubungan keluarga: ")
                    new_contact = FamilyContact(name, phone, email, relationship)
                else:
                    new_contact = Contact(name, phone, email)
                manager.add_contact(new_contact)
            except ValueError as e:
                print(f"Error input: {e}")
            except Exception as e:
                print(f"Terjadi kesalahan tidak terduga: {e}")

        elif choice == '2':
            manager.view_contacts()

        elif choice == '3':
            query = input("Masukkan nama, telepon, atau email untuk dicari: ")
            manager.find_contact(query)

        elif choice == '4':
            contact_id = input("Masukkan ID kontak yang ingin dihapus: ")
            manager.delete_contact(contact_id)

        elif choice == '5':
            print("Terima kasih telah menggunakan Sistem Manajemen Kontak.")
            break
        else:
            print("Pilihan tidak valid. Silakan coba lagi.")

if __name__ == "__main__":
    main()


Kontak berhasil dimuat dari 'my_contacts.json'.

--- Menu Manajemen Kontak ---
1. Tambah Kontak
2. Lihat Semua Kontak
3. Cari Kontak
4. Hapus Kontak
5. Keluar
-----------------------------
Pilih opsi (1-5): 1

--- Tambah Kontak Baru ---
Nama: Christine 
Telepon: 0879
Email: chris@happy.com
Tipe kontak (biasa/keluarga): biasa
Kontak berhasil disimpan ke 'my_contacts.json'.
Kontak 'Christine ' berhasil ditambahkan.

--- Menu Manajemen Kontak ---
1. Tambah Kontak
2. Lihat Semua Kontak
3. Cari Kontak
4. Hapus Kontak
5. Keluar
-----------------------------
Pilih opsi (1-5): 1

--- Tambah Kontak Baru ---
Nama: meong
Telepon: 0987
Email: meong@paw.com
Tipe kontak (biasa/keluarga): keluarga
Hubungan keluarga: kucing
Kontak berhasil disimpan ke 'my_contacts.json'.
Kontak 'meong' berhasil ditambahkan.

--- Menu Manajemen Kontak ---
1. Tambah Kontak
2. Lihat Semua Kontak
3. Cari Kontak
4. Hapus Kontak
5. Keluar
-----------------------------
Pilih opsi (1-5): 2

--- Daftar Kontak ---
ID: C-512129,